In [2]:
import pandas as pd

zero_shot_data = pd.read_csv('zero_shot_answers.csv')
chain_of_thought_data = pd.read_csv('chain_of_thought_answers.csv')

In [3]:
zero_shot_data.head()

,gemma,phi,llama
0,Option 3: k = n = 1\n\nThe equation has one re...,Option 2: k = 1 and n = 0,Option 2: k = 1 and n = 0
1,Option 3: 2\n\nThe property holds for any x in...,2,The answer is Option 2: 1
2,Option 2: n = 1 and r = 7\n\nThe null space n ...,Option 2: n = 1 and r = 7\n\nExplanation: The,The answer is Option 4: n = 2 and r = 6
3,Option 1: 4\n\nThe curve xy = 8 is a line segm...,2sqrt(2),The answer is Option 4: 2sqrt(2)
4,Option 3: 2/23\n\nThe probability of selecting...,2/69,Option 4: 12/125


In [7]:
# create a new df match the regex pattern r'Option [1-9A-Za-z]' and extract the option number

zero_shot_data['gemma_option'] = zero_shot_data['gemma'].str.extract(r'Option ([1-9A-Za-z])')
zero_shot_data['phi_option'] = zero_shot_data['phi'].str.extract(r'Option ([1-9A-Za-z])')
zero_shot_data['llama_option'] = zero_shot_data['llama'].str.extract(r'Option ([1-9A-Za-z])')

In [10]:
dataset = pd.read_parquet("hf://datasets/cais/mmlu/college_mathematics/test-00000-of-00001.parquet")

In [13]:
# increase the value in answer column by 1 to match the option number
dataset['answer'] = dataset['answer'] + 1

In [14]:
zero_shot_data['correct_option'] = dataset['answer']

In [15]:
zero_shot_data

,gemma,phi,llama,gemma_option,phi_option,llama_option,correct_option
0,Option 3: k = n = 1\n\nThe equation has one re...,Option 2: k = 1 and n = 0,Option 2: k = 1 and n = 0,3,2,2,2
1,Option 3: 2\n\nThe property holds for any x in...,2,The answer is Option 2: 1,3,NaN,2,4
2,Option 2: n = 1 and r = 7\n\nThe null space n ...,Option 2: n = 1 and r = 7\n\nExplanation: The,The answer is Option 4: n = 2 and r = 6,2,2,4,4
3,Option 1: 4\n\nThe curve xy = 8 is a line segm...,2sqrt(2),The answer is Option 4: 2sqrt(2),1,NaN,4,1
4,Option 3: 2/23\n\nThe probability of selecting...,2/69,Option 4: 12/125,3,NaN,4,3
...,...,...,...,...,...,...,...
95,Option 3: 0 or 1\n\nThe intersection of two su...,Option 3: 0 or 1,Option 3: 0 or 1,3,3,3,4
96,The answer is Option 1: closed.\n\nA subset S ...,closed,Option 4: totally disconnected,1,NaN,4,3
97,Option 3: x^2 + y^2 = 9\n\nThe equation x^2,Option 4: x^2 + y^2 = 16,Option 4: x^2 + y^2 = 16,3,4,4,3
98,Option 3: 28\n\nThe property states that the n...,Option 4: 35,The answer is Option 1: 24,3,4,1,4


In [24]:
# create a new df for options only
options = zero_shot_data[['gemma_option', 'phi_option', 'llama_option', 'correct_option']]
options

,gemma_option,phi_option,llama_option,correct_option
0,3,2,2,2
1,3,NaN,2,4
2,2,2,4,4
3,1,NaN,4,1
4,3,NaN,4,3
...,...,...,...,...
95,3,3,3,4
96,1,NaN,4,3
97,3,4,4,3
98,3,4,1,4


In [25]:
# replace nan values with 0
options = options.fillna(0)
options

,gemma_option,phi_option,llama_option,correct_option
0,3,2,2,2
1,3,0,2,4
2,2,2,4,4
3,1,0,4,1
4,3,0,4,3
...,...,...,...,...
95,3,3,3,4
96,1,0,4,3
97,3,4,4,3
98,3,4,1,4


In [20]:
options.dropna(inplace=True)
options

/tmp/ipykernel_2314835/2369890251.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  options.dropna(inplace=True)


,gemma_option,phi_option,llama_option,correct_option
0,3,2,2,2
2,2,2,4,4
5,3,4,4,4
6,1,3,1,3
7,3,1,1,3
...,...,...,...,...
94,4,4,4,4
95,3,3,3,4
97,3,4,4,3
98,3,4,1,4


In [26]:
# convert the options to int
options = options.astype(int)

In [27]:
# calculate the accuracy of each model
gemma_accuracy = (options['gemma_option'] == options['correct_option']).mean()
phi_accuracy = (options['phi_option'] == options['correct_option']).mean()
llama_accuracy = (options['llama_option'] == options['correct_option']).mean()

gemma_accuracy, phi_accuracy, llama_accuracy

(0.25, 0.28, 0.35)

In [28]:
chain_of_thought_data['gemma_option'] = chain_of_thought_data['gemma'].str.extract(r'Option ([1-9A-Za-z])')
chain_of_thought_data['phi_option'] = chain_of_thought_data['phi'].str.extract(r'Option ([1-9A-Za-z])')
chain_of_thought_data['llama_option'] = chain_of_thought_data['llama'].str.extract(r'Option ([1-9A-Za-z])')

In [31]:
chain_of_thought_data['correct_option'] = dataset['answer']

In [32]:
# create a new df for options only
options = chain_of_thought_data[['gemma_option', 'phi_option', 'llama_option', 'correct_option']]
options

,gemma_option,phi_option,llama_option,correct_option
0,3,2,2,2
1,NaN,1,NaN,4
2,2,1,NaN,4
3,3,1,NaN,1
4,3,3,NaN,3
...,...,...,...,...
95,3,3,NaN,4
96,1,1,1,3
97,3,1,1,3
98,4,NaN,4,4


In [33]:
# replace nan values with 0
options = options.fillna(0)

In [34]:
# convert the options to int
options = options.astype(int)

In [35]:
# calculate the accuracy of each model
gemma_accuracy = (options['gemma_option'] == options['correct_option']).mean()
phi_accuracy = (options['phi_option'] == options['correct_option']).mean()
llama_accuracy = (options['llama_option'] == options['correct_option']).mean()

gemma_accuracy, phi_accuracy, llama_accuracy

(0.27, 0.36, 0.3)